### Association rules by Department Bakery
#### Department - Bakery

In [2]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [3]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [4]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [5]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [6]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [7]:
combine_train_prior

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,1,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods
2,2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,fresh vegetables,produce
3,3,1,22035,8,1,112108,train,4,4,10,9.0,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs
4,4,1,47209,7,0,112108,train,4,4,10,9.0,Organic Hass Avocado,24,4,fresh fruits,produce
5,5,1,13176,6,0,112108,train,4,4,10,9.0,Bag of Organic Bananas,24,4,fresh fruits,produce
6,6,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
7,7,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,fresh vegetables,produce
8,8,36,46979,8,1,79431,train,23,6,18,30.0,Asparagus,83,4,fresh vegetables,produce
9,9,36,39612,1,0,79431,train,23,6,18,30.0,Grated Pecorino Romano Cheese,2,16,specialty cheeses,dairy eggs


In [8]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="bakery"].reset_index()

In [9]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [10]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=4)

In [11]:
grouped_data

,order_id,product_name
9,1983,Everything Bagels
10,1983,Rocky Mountain Sourdough Bread
11,1983,Organic Honey Sweet Whole Wheat Bread
12,1983,Plain Bagels
13,1983,100% Whole Wheat Bread
14,1983,"Muffins, Flax, Wild Blueberry"
52,4309,Plain Mini Bagels
53,4309,Soft Taco Size Flour Tortillas 10 Count
54,4309,Country White Bread
55,4309,Original Cinnamon Bread


In [12]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [13]:
#relace NAN with 0
table1=table1.fillna(0)

In [14]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1


In [15]:
basket_sets = table1.applymap(encode_units)

In [16]:
## Running Apriori to get frequent itemset with a minimum support of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.009, use_colnames=True)

In [17]:
frequent_itemsets

,support,itemsets
0,0.020795,[100% Whole Wheat Bagels]
1,0.143356,[100% Whole Wheat Bread]
2,0.010326,[100% Whole Wheat Medium Soft Taco Flour Torti...
3,0.035892,[100% Whole Wheat Pre-Sliced Bagels]
4,0.009756,[100% Whole Wheat Slice English Muffins]
5,0.009899,[12 Grain Bread]
6,0.016664,[Authentic French Brioche]
7,0.020154,[Authentic French Brioche Hamburger Buns]
8,0.017519,[Bagel Thins Everything]
9,0.014314,[Best Cinnamon Bread]


In [18]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [19]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Original Nooks & Crannies English Muffins),(100% Whole Wheat Bread),0.095286,0.143356,0.015311,0.160688,1.120902,0.001651,1.020650
1,(100% Whole Wheat Bread),(Original Nooks & Crannies English Muffins),0.143356,0.095286,0.015311,0.106806,1.120902,0.001651,1.012898
2,(Deluxe Everything Bagels),(Deluxe Blueberry Bagels),0.037530,0.027489,0.012890,0.343454,12.494234,0.011858,1.481252
3,(Deluxe Blueberry Bagels),(Deluxe Everything Bagels),0.027489,0.037530,0.012890,0.468912,12.494234,0.011858,1.812260
4,"(Sliced Blueberry Bagels, Presliced Everything...",(Plain Pre-Sliced Bagels),0.017092,0.079333,0.011181,0.654167,8.245788,0.009825,2.662168
5,"(Sliced Blueberry Bagels, Plain Pre-Sliced Bag...",(Presliced Everything Bagels),0.014029,0.077339,0.011181,0.796954,10.304634,0.010096,4.544103
6,"(Presliced Everything Bagels, Plain Pre-Sliced...",(Sliced Blueberry Bagels),0.030978,0.024498,0.011181,0.360920,14.732652,0.010422,1.526415
7,(Sliced Blueberry Bagels),"(Presliced Everything Bagels, Plain Pre-Sliced...",0.024498,0.030978,0.011181,0.456395,14.732652,0.010422,1.782585
8,(Presliced Everything Bagels),"(Sliced Blueberry Bagels, Plain Pre-Sliced Bag...",0.077339,0.014029,0.011181,0.144567,10.304634,0.010096,1.152599
9,(Plain Pre-Sliced Bagels),"(Sliced Blueberry Bagels, Presliced Everything...",0.079333,0.017092,0.011181,0.140934,8.245788,0.009825,1.144159


In [20]:
## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Original Nooks & Crannies English Muffins),(100% Whole Wheat Bread),0.095286,0.143356,0.015311,0.160688,1.120902,0.001651,1.020650
1,(100% Whole Wheat Bread),(Original Nooks & Crannies English Muffins),0.143356,0.095286,0.015311,0.106806,1.120902,0.001651,1.012898
2,(Deluxe Everything Bagels),(Deluxe Blueberry Bagels),0.037530,0.027489,0.012890,0.343454,12.494234,0.011858,1.481252
3,(Deluxe Blueberry Bagels),(Deluxe Everything Bagels),0.027489,0.037530,0.012890,0.468912,12.494234,0.011858,1.812260
4,"(Sliced Blueberry Bagels, Presliced Everything...",(Plain Pre-Sliced Bagels),0.017092,0.079333,0.011181,0.654167,8.245788,0.009825,2.662168
5,"(Sliced Blueberry Bagels, Plain Pre-Sliced Bag...",(Presliced Everything Bagels),0.014029,0.077339,0.011181,0.796954,10.304634,0.010096,4.544103
6,"(Presliced Everything Bagels, Plain Pre-Sliced...",(Sliced Blueberry Bagels),0.030978,0.024498,0.011181,0.360920,14.732652,0.010422,1.526415
7,(Sliced Blueberry Bagels),"(Presliced Everything Bagels, Plain Pre-Sliced...",0.024498,0.030978,0.011181,0.456395,14.732652,0.010422,1.782585
8,(Presliced Everything Bagels),"(Sliced Blueberry Bagels, Plain Pre-Sliced Bag...",0.077339,0.014029,0.011181,0.144567,10.304634,0.010096,1.152599
9,(Plain Pre-Sliced Bagels),"(Sliced Blueberry Bagels, Presliced Everything...",0.079333,0.017092,0.011181,0.140934,8.245788,0.009825,1.144159


In [21]:
final_result.to_csv("Department_BakeryApriori.csv",sep=',')

In [22]:
test1=pd.read_csv("Department_BakeryApriori.csv")

In [23]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [24]:
test1=test1.replace({"u'": ''}, regex=True)

In [25]:
test1['antecedants'] = test1['antecedants'].str.strip('()').str.strip('[]')
test1['antecedants'] = test1['antecedants'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['antecedants'] = test1['antecedants'].str.replace("'",'')

In [26]:
test1['consequents'] = test1['consequents'].str.strip('()')
test1['consequents'] = test1['consequents'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['consequents'] = test1['consequents'].str.replace("'",'')

In [27]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [28]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Original Nooks & Crannies English Muffins,100% Whole Wheat Bread,0.095286,0.143356,0.015311,0.160688,1.120902,0.001651,1.020650
1,100% Whole Wheat Bread,Original Nooks & Crannies English Muffins,0.143356,0.095286,0.015311,0.106806,1.120902,0.001651,1.012898
2,Deluxe Everything Bagels,Deluxe Blueberry Bagels,0.037530,0.027489,0.012890,0.343454,12.494234,0.011858,1.481252
3,Deluxe Blueberry Bagels,Deluxe Everything Bagels,0.027489,0.037530,0.012890,0.468912,12.494234,0.011858,1.812260
4,"Sliced Blueberry Bagels, Presliced Everything ...",Plain Pre-Sliced Bagels,0.017092,0.079333,0.011181,0.654167,8.245788,0.009825,2.662168
5,"Sliced Blueberry Bagels, Plain Pre-Sliced Bagels",Presliced Everything Bagels,0.014029,0.077339,0.011181,0.796954,10.304634,0.010096,4.544103
6,"Presliced Everything Bagels, Plain Pre-Sliced ...",Sliced Blueberry Bagels,0.030978,0.024498,0.011181,0.360920,14.732652,0.010422,1.526415
7,Sliced Blueberry Bagels,"Presliced Everything Bagels, Plain Pre-Sliced ...",0.024498,0.030978,0.011181,0.456395,14.732652,0.010422,1.782585
8,Presliced Everything Bagels,"Sliced Blueberry Bagels, Plain Pre-Sliced Bagels",0.077339,0.014029,0.011181,0.144567,10.304634,0.010096,1.152599
9,Plain Pre-Sliced Bagels,"Sliced Blueberry Bagels, Presliced Everything ...",0.079333,0.017092,0.011181,0.140934,8.245788,0.009825,1.144159


In [29]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_BakeryApriori.csv")

In [30]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Original Nooks & Crannies English Muffins,100% Whole Wheat Bread,0.095286,0.143356,0.015311,0.160688,1.120902,0.001651,1.020650
1,100% Whole Wheat Bread,Original Nooks & Crannies English Muffins,0.143356,0.095286,0.015311,0.106806,1.120902,0.001651,1.012898
2,Deluxe Everything Bagels,Deluxe Blueberry Bagels,0.037530,0.027489,0.012890,0.343454,12.494234,0.011858,1.481252
3,Deluxe Blueberry Bagels,Deluxe Everything Bagels,0.027489,0.037530,0.012890,0.468912,12.494234,0.011858,1.812260
4,"Sliced Blueberry Bagels, Presliced Everything ...",Plain Pre-Sliced Bagels,0.017092,0.079333,0.011181,0.654167,8.245788,0.009825,2.662168
5,"Sliced Blueberry Bagels, Plain Pre-Sliced Bagels",Presliced Everything Bagels,0.014029,0.077339,0.011181,0.796954,10.304634,0.010096,4.544103
6,"Presliced Everything Bagels, Plain Pre-Sliced ...",Sliced Blueberry Bagels,0.030978,0.024498,0.011181,0.360920,14.732652,0.010422,1.526415
7,Sliced Blueberry Bagels,"Presliced Everything Bagels, Plain Pre-Sliced ...",0.024498,0.030978,0.011181,0.456395,14.732652,0.010422,1.782585
8,Presliced Everything Bagels,"Sliced Blueberry Bagels, Plain Pre-Sliced Bagels",0.077339,0.014029,0.011181,0.144567,10.304634,0.010096,1.152599
9,Plain Pre-Sliced Bagels,"Sliced Blueberry Bagels, Presliced Everything ...",0.079333,0.017092,0.011181,0.140934,8.245788,0.009825,1.144159


In [31]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()